### Load Damagescanner package

In [2]:
import os,sys
os.environ['USE_PYGEOS'] = '0'
import rasterio
import pyproj
import shapely
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from pathlib import Path,PurePath

sys.path.append("../src/damagescanner")

data_path = Path('..')

### Read input data

In [3]:
inun_map = data_path / 'data' / 'hazard' / 'inundation_map.tif'
landuse_map =  data_path / 'data' / 'landuse' / 'landuse.shp'

### Create dummy maximum damage dictionary and curves DataFrame

In [4]:
maxdam = {"grass":5,
        "forest":10,
        "orchard":50,
        "residential":200,
        "industrial":300,
        "retail":300,
        "farmland":10,
        "cemetery":15,
        "construction":10,
        "meadow":5,
        "farmyard":5,
        "scrub":5,
        "allotments":10,
        "reservoir":5,
        "static_caravan":100,
        "commercial":300}

curves = np.array(
            [[0,0],
            [50,0.2],
            [100,0.4],
            [150,0.6],
            [200,0.8],
            [250,1]])  
    
curves = np.concatenate((curves,
                            np.transpose(np.array([curves[:,1]]*(len(maxdam)-1)))),
                           axis=1)

curves = pd.DataFrame(curves)
curves.columns = ['depth']+list(maxdam.keys())
curves.set_index('depth',inplace=True)    

In [5]:
curves

,grass,forest,orchard,residential,industrial,retail,farmland,cemetery,construction,meadow,farmyard,scrub,allotments,reservoir,static_caravan,commercial
depth,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
100.0,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
150.0,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6
200.0,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8
250.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Run the VectorScanner

In [6]:
%%time
lu_damage = VectorScanner(landuse_map,
                            inun_map,
                            curves,
                            maxdam,
                            cell_size = 5,
                            exp_crs = 28992,
                            haz_crs = 28992,
                            object_col='landuse',
                            hazard_col='inun_val',
                            centimeters=False,
                            save=False)

NameError: name 'VectorScanner' is not defined

In [8]:
curves['grass']

depth
0.0      0.0
50.0     0.2
100.0    0.4
150.0    0.6
200.0    0.8
250.0    1.0
Name: grass, dtype: float64